In [123]:
from clarifai.rest import ClarifaiApp
import json
from clarifai.rest import Image as ClImage
import numpy as np
import matplotlib.pylab as plt

In [124]:
app = ClarifaiApp(api_key='657ff2fc61da403c8e9989dcf5dcaa5a')

In [8]:
#General Embedding model
model = app.models.get(model_id="bbb5f41425b8468d9b7a554ff10f8581")
response = model.predict_by_url(url='https://samples.clarifai.com/metro-north.jpg')

In [125]:
with open('D:/Lnmwdnq3YcF7F3YsJncp/data/train.jsonl', 'r') as json_file:
    json_list = list(json_file)
    
label_train = []
img_link_train = []
text_train = []
for json_str in json_list:
    result = json.loads(json_str)
    label_train.append(result['label'])
    img_link_train.append(result['img'])
    text_train.append(result['text'])

In [126]:
labels = np.asarray(label_train)
print(len(labels[labels == 0]) ,  len(labels[labels == 1]))

5450 3050


In [127]:
N = 5000

images = []
for file, lb in zip(img_link_train[:N], label_train[:N]):
    if lb == 0:
        c = 'Not offensive'
    else:
        c = 'Offensive'
    img = ClImage(filename='D:/Lnmwdnq3YcF7F3YsJncp/data/' + file, concepts=[c])
    images.append(img)
    if len(images) == 100:
        app.inputs.bulk_create_images(images)
        images = []

In [128]:
model = app.models.create(model_id="meme_v1", concepts=["Offensive", "Not offensive"])

In [129]:
model.train()

In [130]:
model = app.models.get('meme_v1')

label_test = []
img_link_test= []
text_test = []

for json_str in json_list[1000:1100]:
    result = json.loads(json_str)
    
    label_test.append(result['label'])
    img_link_test.append(result['img'])
    text_test.append(result['text'])


pred = []
for file in img_link_test:
    res = model.predict_by_filename(filename='D:/Lnmwdnq3YcF7F3YsJncp/data/' + file)
    for i in range(2):
        if res['outputs'][0]['data']['concepts'][i]['id'] == 'Offensive':
            pred.append(res['outputs'][0]['data']['concepts'][i]['value'])

In [131]:
label_test = np.asarray(label_test)
pred = np.asarray(pred)

In [132]:
print(pred[label_test == 0].mean(), pred[label_test == 1].mean())

0.35882549140000003 0.54392962425
